In [1]:
const dfd = require("danfojs-node");
const tf = dfd.tensorflow
// const tf = require('@tensorflow/tfjs');

In [2]:
config = {
  rowStart: 7228,
  rowEnd: 7288,
  timesteps: 2,
  maxBatchSize: 1,
  layerSize: 1,
  unitSize: 1,
  dropout: 0,
  learningRate: 0.1,
  maxEpochs: 50,
  timeCol: 'tanggal',
  feature: ['rr']
}

// usecols = [config.timeCol].concat(config.feature)

function trainTestSplit(dataset, timeStep = 1) {
  dataX = []
  dataY = []

  // for (let i = 0; i < dataset.values.length - timeStep - 1; i++) {
  for (let i = 0; i < dataset.values.length - timeStep; i++) {
    dataX.push(dataset.iloc({ rows: [`${i}:${i + timeStep}`] }).values)
    dataY.push(dataset.iloc({ rows: [i + timeStep] }).values[0])
  }

  return [dataX, dataY]
}

{
  rowStart: 7228,
  rowEnd: 7288,
  timesteps: 2,
  maxBatchSize: 1,
  layerSize: 1,
  unitSize: 1,
  dropout: 0,
  learningRate: 0.1,
  maxEpochs: 50,
  timeCol: 'tanggal',
  feature: [ 'rr' ]
}

In [3]:
$$async$$ = true;
(async function () {
  dataset = await dfd.readCSV("../Data/1985-2021.csv")
  dataset = dataset.iloc({rows:[`${config.rowStart}:${config.rowEnd}`]})
  datelist = dataset.loc({columns: [config.timeCol]})

  $$done$$(dataset.shape)
})()

[ 60, 11 ]

In [4]:
featureset = dataset.loc({columns: config.feature})

scaller = new dfd.MinMaxScaler()
scaller.fit(featureset)

featuresetScaled = scaller.transform(featureset)

trainSize = featuresetScaled.values.length * 0.9
trainset = featuresetScaled.iloc({rows: [`${0}:${trainSize}`]})
testset =  featuresetScaled.iloc({rows: [`${trainSize}:${featuresetScaled.values.length}`]})

trainset = trainTestSplit(trainset, timeStep=config.timesteps)
testset = trainTestSplit(testset, timeStep=config.timesteps)

XTrain = tf.tensor(trainset[0])
yTrain = tf.tensor(trainset[1])

XTest = tf.tensor(testset[0])
yTest = tf.tensor(testset[1])

XTrain = tf.reshape(XTrain, [XTrain.shape[0], config.feature.length, XTrain.shape[1]])
XTest = tf.reshape(XTest, [XTest.shape[0], config.feature.length, XTest.shape[1]])

Tensor {
  kept: false,
  isDisposedInternal: false,
  shape: [ 4, 1, 2 ],
  dtype: 'float32',
  size: 8,
  strides: [ 2, 2 ],
  dataId: {},
  id: 18,
  rankType: '3',
  scopeId: 6
}

In [5]:
// yPredLoss = []
// yTrueLoss = []
// loss = []

// function meanSquaredError(yTrue, yPred) {
//   yPredLoss.push(yPred.array())
//   yTrueLoss.push(yTrue.array())
//   loss.push(tf.losses.meanSquaredError(yTrue, yPred).arraySync())
//   return tf.losses.meanSquaredError(yTrue, yPred)
// }

model = tf.sequential()

model.add(tf.layers.lstm({
  units: config.unitSize,
  batchInputShape: [config.maxBatchSize, config.feature.length, config.timesteps],
  goBackwards: true,
  dropout: config.dropout,
  weights: [
    tf.tensor([
      [0.5774, 0.5774, 0.5774, 0.5774],
      [0.5774, 0.5774, 0.5774, 0.5774]
    ]),
    tf.tensor([
      [0.5774, 0.5774, 0.5774, 0.5774]
    ]),
    tf.zeros([4])
  ]
}))

// model.layers[0].getWeights()[0].print()

model.compile({
  optimizer: tf.train.sgd(config.learningRate),
  loss: 'meanSquaredError',
  // loss: meanSquaredError,
  runEagerly: true
})

model.summary()

__________________________________________________________________________________________
Layer (type)                Input Shape               Output shape              Param #   
lstm_LSTM1 (LSTM)           [[1,1,2]]                 [1,1]                     16        
Total params: 16
Trainable params: 16
Non-trainable params: 0
__________________________________________________________________________________________


In [ ]:
// batchEnd = []
epochEnd = []

$$async$$ = true;
(async function () {
  history = await model.fit(XTrain, yTrain, {
    shuffle: false,
    epochs: config.maxEpochs,
    verbose: 1,
    batchSize: config.maxBatchSize,
    callbacks: [
      new tf.CustomCallback({
        // onEpochBegin: null,
        onEpochEnd: (epoch, logs = null) => epochEnd.push(logs.loss),
        // on_batch_begin: null,
        // onBatchEnd: (batch, logs = null) => batchEnd.push(logs.loss),
        // onTrainBegin: null,
        // onTrainEnd: null,
      })
    ]
  })
  $$done$$()
})()

In [7]:
// $$async$$ = true;
// (async function () {
//   await Promise.all(yPredLoss).then(values => {
//     console.log(values);
//   });
//   $$done$$()
// })()

In [9]:
model.predict(XTest, yTest).dataSync()

Float32Array(4) [
  0.1997341513633728,
  0.187837153673172,
  0.18746259808540344,
  0.18775293231010437
]

In [16]:
testBatchEnd = []

model.evaluate(XTest, yTest, {
  callbacks: [
    new tf.CustomCallback({
      onBatchEnd: (batch, logs = null) => testBatchEnd.push(logs.loss),
    })
  ]
}).dataSync()

Float32Array(1) [ 0.03289681673049927 ]

In [17]:
testBatchEnd

[]